Anlalyze times and hits of articles to upload 
a article in a timely manner to find a mentor

In [282]:
# 기본 사용법 (pip install beautifulsoup4)
from bs4 import BeautifulSoup
# from 모듈이름 import 클래스 이름
import urllib.request as req
import pandas as pd
import time

In [234]:
url = 'https://www.clien.net/service/group/community?&od=T31&po=0'
res = req.urlopen(url) #소스를 res로 넣는다
soup = BeautifulSoup(res, "html.parser")

In [151]:
recommend = soup.select("div.list_content > div > div.list_symph > span")
category = soup.select("div > div.list_title > a > span.shortname.fixed")
title = soup.select("div > div.list_title > a > span.subject_fixed")
reply = soup.select("div.list_content > div > div.list_title > a")
writer = soup.select(" div.list_content > div > div.list_author")
viewCnt = soup.select("div.list_content > div > div.list_hit > span ")
dateTime =soup.select("div.list_content > div > div.list_time > span")
docNo = soup.select("div.list_content > div > div.list_title > a.list_subject")

In [214]:
# 라이터가 이미지인 경우에
# 리플이 안달린 경우에
reply = soup.select("div.list_content > div > div.list_title > a")
reply[1].text # try except
# 에러  나면 0으로 추가하도록

'\n4\n\n\n'

In [195]:

writer = soup.select(" div.list_content > div > div.list_author")

str(writer[28]).split('img alt="')[1].split('"')[0]


len(writer[2].text.split("\n")[3])
#그림 없는경우
writer[2].text.split("\n")[3]
writer[28].text.split("\n")[3]

# 그림이 있는 경우는 len이 0 이경우에는 
str(writer[28]).split('img alt="')[1].split('"')[0]


'초코클랸'

In [166]:
def readPage(firstPage,lastPage)

    info=pd.DataFrame()

    for i in range(firstPage,lastPage):
        url = "https://www.clien.net/service/group/community?&od=T31&po=" + str(i)
        res = req.urlopen(url) #소스를 res로 넣는다
        soup = BeautifulSoup(res, "html.parser")
        
        recommend = soup.select("div.list_content > div > div.list_symph > span")
        category = soup.select("div > div.list_title > a > span.shortname.fixed")
        title = soup.select("div > div.list_title > a > span.subject_fixed")
        reply = soup.select("div.list_content > div > div.list_title > a")
        writer = soup.select(" div.list_content > div > div.list_author")
        viewCnt = soup.select("div.list_content > div > div.list_hit > span ")
        dateTime =soup.select("div.list_content > div > div.list_time > span")
        docNo = soup.select("div.list_content > div > div.list_title > a.list_subject")
        
        recList =[]
        catList =[]
        titList =[]
        repList =[]
        wriList =[]
        viewList =[]
        datList =[]
        docList = []
        
        for ind in range(30):
            rec = recommend[ind].text
            cat = category[ind].text
            tit = title[ind].text      
            view = viewCnt[ind].text
            dat = dateTime[ind].text.split("\t")[13]
            doc = doc[ind]["href"].split("/")[4].split("?")[0]
            
            try:
                int(reply[ind].text)
                rep = reply[ind].text
            except:
                rep = 0
                
            if len(writer[ind].text.split("\n")[3]) == 0:
                wri = str(writer[ind]).split('img alt="')[1].split('"')[0]
            else:
                wri = writer[ind].text.split("\n")[3]
                
            recList.append(rec)
            catList.append(cat)
            titList.append(tit)
            repList.append(rep)
            wriList.append(wri)
            viewList.append(view)
            datList.append(dat)
            docList.append(doc)
            
        # list into dataframe
        info = pd.DataFrame(columns['추천수'],data = recList)
        info['게시판'] = catList
        info['제목'] = titList
        info['리플수'] = repList
        info['아이디'] = wriList
        info['조회수'] = viewList
        info['날짜'] = datList
        info['문서번호'] = docList

    accum = pd.concat([accum,info])
    info =pd.DataFrame()
    
accum.to('/list.csv',header=False)

SyntaxError: invalid syntax (<ipython-input-166-579d255a9752>, line 1)

In [ ]:
info=pd.DataFrame()
accum=pd.DataFrame()

for i in range(0,1800):
    url = "https://www.clien.net/service/group/community?&od=T31&po=" + str(i)
    print(url)
    res = req.urlopen(url) #소스를 res로 넣는다
    soup = BeautifulSoup(res, "html.parser")

    recommend = soup.select("div.list_content > div > div.list_symph > span")
    category = soup.select("div > div.list_title > a > span.shortname.fixed")
    title = soup.select("div > div.list_title > a > span.subject_fixed")
    reply = soup.select("div.list_content > div > div.list_title > a")
    writer = soup.select(" div.list_content > div > div.list_author")
    viewCnt = soup.select("div.list_content > div > div.list_hit > span ")
    dateTime =soup.select("div.list_content > div > div.list_time > span")
    docNo = soup.select("div.list_content > div > div.list_title > a.list_subject")
    photoyn = soup.select("div.list_content > div > div.list_title")


    recList =[]
    catList =[]
    titList =[]
    repList =[]
    wriList =[]
    viewList =[]
    datList =[]
    docList = []
    phoList = []

    
    # 가끔 게시물이 30개가 아닌경우가 있음
    length=len(recommend)

    for ind in range(length):
        rec = recommend[ind].text
        cat = category[ind].text
        tit = title[ind].text      
        view = viewCnt[ind].text
        dat = dateTime[ind].text.split("\t")[13].rstrip('\n')
        doc = docNo[ind]["href"].split("/")[4].split("?")[0]
        
        # if there's photo in article, also class "icon"
        if len(str(photoyn[0]).split("icon"))==2:
            pho = "Y"
        else:
            pho = "N"
        
        # if there's no reply, no data
        try:
            int(reply[ind].text)
            rep = int(reply[ind].text)
        except:
            rep = 0

        # if someone's nickname is image
        
        if len(writer[ind].text.split("\n")[3]) == 0:
            wri = str(writer[ind]).split('img alt="')[1].split('"')[0]
        else:
            wri = writer[ind].text.split("\n")[3]

        recList.append(rec)
        catList.append(cat)
        titList.append(tit)
        repList.append(rep)
        wriList.append(wri)
        viewList.append(view)
        datList.append(dat)
        docList.append(doc)
        phoList.append(pho)

    # list into dataframe
    info = pd.DataFrame(columns=['추천수'],data = recList)
    info['게시판'] = catList
    info['제목'] = titList
    info['리플수'] = repList
    info['아이디'] = wriList
    info['조회수'] = viewList
    info['날짜'] = datList
    info['문서번호'] = docList
    info['사진유무'] = phoList
    if len(info) != 30:
        print("길이오류")
    
    accum = pd.concat([accum,info])
    info =pd.DataFrame()
    
    time.sleep(1)

https://www.clien.net/service/group/community?&od=T31&po=0
https://www.clien.net/service/group/community?&od=T31&po=1
https://www.clien.net/service/group/community?&od=T31&po=2
https://www.clien.net/service/group/community?&od=T31&po=3
https://www.clien.net/service/group/community?&od=T31&po=4
https://www.clien.net/service/group/community?&od=T31&po=5
https://www.clien.net/service/group/community?&od=T31&po=6
https://www.clien.net/service/group/community?&od=T31&po=7
https://www.clien.net/service/group/community?&od=T31&po=8
https://www.clien.net/service/group/community?&od=T31&po=9
https://www.clien.net/service/group/community?&od=T31&po=10
https://www.clien.net/service/group/community?&od=T31&po=11
https://www.clien.net/service/group/community?&od=T31&po=12
https://www.clien.net/service/group/community?&od=T31&po=13
https://www.clien.net/service/group/community?&od=T31&po=14
https://www.clien.net/service/group/community?&od=T31&po=15
https://www.clien.net/service/group/community?&od=

길이오류
https://www.clien.net/service/group/community?&od=T31&po=136
https://www.clien.net/service/group/community?&od=T31&po=137
https://www.clien.net/service/group/community?&od=T31&po=138
https://www.clien.net/service/group/community?&od=T31&po=139
https://www.clien.net/service/group/community?&od=T31&po=140
https://www.clien.net/service/group/community?&od=T31&po=141
https://www.clien.net/service/group/community?&od=T31&po=142
https://www.clien.net/service/group/community?&od=T31&po=143
https://www.clien.net/service/group/community?&od=T31&po=144
https://www.clien.net/service/group/community?&od=T31&po=145
https://www.clien.net/service/group/community?&od=T31&po=146
https://www.clien.net/service/group/community?&od=T31&po=147
https://www.clien.net/service/group/community?&od=T31&po=148
https://www.clien.net/service/group/community?&od=T31&po=149
https://www.clien.net/service/group/community?&od=T31&po=150
https://www.clien.net/service/group/community?&od=T31&po=151
https://www.clien.n

https://www.clien.net/service/group/community?&od=T31&po=270
https://www.clien.net/service/group/community?&od=T31&po=271
https://www.clien.net/service/group/community?&od=T31&po=272
https://www.clien.net/service/group/community?&od=T31&po=273
https://www.clien.net/service/group/community?&od=T31&po=274
https://www.clien.net/service/group/community?&od=T31&po=275
https://www.clien.net/service/group/community?&od=T31&po=276
https://www.clien.net/service/group/community?&od=T31&po=277
길이오류
https://www.clien.net/service/group/community?&od=T31&po=278
https://www.clien.net/service/group/community?&od=T31&po=279
https://www.clien.net/service/group/community?&od=T31&po=280
https://www.clien.net/service/group/community?&od=T31&po=281
https://www.clien.net/service/group/community?&od=T31&po=282
https://www.clien.net/service/group/community?&od=T31&po=283
https://www.clien.net/service/group/community?&od=T31&po=284
https://www.clien.net/service/group/community?&od=T31&po=285
https://www.clien.n

https://www.clien.net/service/group/community?&od=T31&po=404
https://www.clien.net/service/group/community?&od=T31&po=405
https://www.clien.net/service/group/community?&od=T31&po=406
https://www.clien.net/service/group/community?&od=T31&po=407
https://www.clien.net/service/group/community?&od=T31&po=408
https://www.clien.net/service/group/community?&od=T31&po=409
https://www.clien.net/service/group/community?&od=T31&po=410
https://www.clien.net/service/group/community?&od=T31&po=411
https://www.clien.net/service/group/community?&od=T31&po=412
https://www.clien.net/service/group/community?&od=T31&po=413
https://www.clien.net/service/group/community?&od=T31&po=414
https://www.clien.net/service/group/community?&od=T31&po=415
길이오류
https://www.clien.net/service/group/community?&od=T31&po=416
https://www.clien.net/service/group/community?&od=T31&po=417
https://www.clien.net/service/group/community?&od=T31&po=418
https://www.clien.net/service/group/community?&od=T31&po=419
https://www.clien.n

https://www.clien.net/service/group/community?&od=T31&po=538
https://www.clien.net/service/group/community?&od=T31&po=539
https://www.clien.net/service/group/community?&od=T31&po=540
https://www.clien.net/service/group/community?&od=T31&po=541
https://www.clien.net/service/group/community?&od=T31&po=542
길이오류
https://www.clien.net/service/group/community?&od=T31&po=543
https://www.clien.net/service/group/community?&od=T31&po=544
https://www.clien.net/service/group/community?&od=T31&po=545
https://www.clien.net/service/group/community?&od=T31&po=546
https://www.clien.net/service/group/community?&od=T31&po=547
https://www.clien.net/service/group/community?&od=T31&po=548
https://www.clien.net/service/group/community?&od=T31&po=549
https://www.clien.net/service/group/community?&od=T31&po=550
길이오류
https://www.clien.net/service/group/community?&od=T31&po=551
https://www.clien.net/service/group/community?&od=T31&po=552
길이오류
https://www.clien.net/service/group/community?&od=T31&po=553
길이오류
http

...

https://www.clien.net/service/group/community?&od=T31&po=939
길이오류
https://www.clien.net/service/group/community?&od=T31&po=940
https://www.clien.net/service/group/community?&od=T31&po=941
https://www.clien.net/service/group/community?&od=T31&po=942
https://www.clien.net/service/group/community?&od=T31&po=943
https://www.clien.net/service/group/community?&od=T31&po=944
https://www.clien.net/service/group/community?&od=T31&po=945
https://www.clien.net/service/group/community?&od=T31&po=946
길이오류
https://www.clien.net/service/group/community?&od=T31&po=947
https://www.clien.net/service/group/community?&od=T31&po=948
https://www.clien.net/service/group/community?&od=T31&po=949
https://www.clien.net/service/group/community?&od=T31&po=950
https://www.clien.net/service/group/community?&od=T31&po=951
https://www.clien.net/service/group/community?&od=T31&po=952
https://www.clien.net/service/group/community?&od=T31&po=953
https://www.clien.net/service/group/community?&od=T31&po=954
https://www.cl

https://www.clien.net/service/group/community?&od=T31&po=1071
https://www.clien.net/service/group/community?&od=T31&po=1072
https://www.clien.net/service/group/community?&od=T31&po=1073
https://www.clien.net/service/group/community?&od=T31&po=1074
길이오류
https://www.clien.net/service/group/community?&od=T31&po=1075
https://www.clien.net/service/group/community?&od=T31&po=1076
https://www.clien.net/service/group/community?&od=T31&po=1077
https://www.clien.net/service/group/community?&od=T31&po=1078
https://www.clien.net/service/group/community?&od=T31&po=1079
https://www.clien.net/service/group/community?&od=T31&po=1080
https://www.clien.net/service/group/community?&od=T31&po=1081
길이오류
https://www.clien.net/service/group/community?&od=T31&po=1082
https://www.clien.net/service/group/community?&od=T31&po=1083
https://www.clien.net/service/group/community?&od=T31&po=1084
길이오류
https://www.clien.net/service/group/community?&od=T31&po=1085
https://www.clien.net/service/group/community?&od=T31&p

https://www.clien.net/service/group/community?&od=T31&po=1202
https://www.clien.net/service/group/community?&od=T31&po=1203
https://www.clien.net/service/group/community?&od=T31&po=1204
https://www.clien.net/service/group/community?&od=T31&po=1205
길이오류
https://www.clien.net/service/group/community?&od=T31&po=1206
https://www.clien.net/service/group/community?&od=T31&po=1207
https://www.clien.net/service/group/community?&od=T31&po=1208
https://www.clien.net/service/group/community?&od=T31&po=1209
길이오류
https://www.clien.net/service/group/community?&od=T31&po=1210
https://www.clien.net/service/group/community?&od=T31&po=1211
https://www.clien.net/service/group/community?&od=T31&po=1212
https://www.clien.net/service/group/community?&od=T31&po=1213
https://www.clien.net/service/group/community?&od=T31&po=1214
https://www.clien.net/service/group/community?&od=T31&po=1215
https://www.clien.net/service/group/community?&od=T31&po=1216
길이오류
https://www.clien.net/service/group/community?&od=T31&p

https://www.clien.net/service/group/community?&od=T31&po=1333
https://www.clien.net/service/group/community?&od=T31&po=1334
https://www.clien.net/service/group/community?&od=T31&po=1335
https://www.clien.net/service/group/community?&od=T31&po=1336
https://www.clien.net/service/group/community?&od=T31&po=1337
https://www.clien.net/service/group/community?&od=T31&po=1338
https://www.clien.net/service/group/community?&od=T31&po=1339
https://www.clien.net/service/group/community?&od=T31&po=1340
https://www.clien.net/service/group/community?&od=T31&po=1341
https://www.clien.net/service/group/community?&od=T31&po=1342
https://www.clien.net/service/group/community?&od=T31&po=1343
https://www.clien.net/service/group/community?&od=T31&po=1344
https://www.clien.net/service/group/community?&od=T31&po=1345
https://www.clien.net/service/group/community?&od=T31&po=1346
https://www.clien.net/service/group/community?&od=T31&po=1347
https://www.clien.net/service/group/community?&od=T31&po=1348
https://

https://www.clien.net/service/group/community?&od=T31&po=1465
https://www.clien.net/service/group/community?&od=T31&po=1466
https://www.clien.net/service/group/community?&od=T31&po=1467
https://www.clien.net/service/group/community?&od=T31&po=1468
길이오류
https://www.clien.net/service/group/community?&od=T31&po=1469
https://www.clien.net/service/group/community?&od=T31&po=1470
https://www.clien.net/service/group/community?&od=T31&po=1471
https://www.clien.net/service/group/community?&od=T31&po=1472
https://www.clien.net/service/group/community?&od=T31&po=1473
길이오류
https://www.clien.net/service/group/community?&od=T31&po=1474
https://www.clien.net/service/group/community?&od=T31&po=1475
https://www.clien.net/service/group/community?&od=T31&po=1476
https://www.clien.net/service/group/community?&od=T31&po=1477
https://www.clien.net/service/group/community?&od=T31&po=1478
https://www.clien.net/service/group/community?&od=T31&po=1479
https://www.clien.net/service/group/community?&od=T31&po=148

In [295]:
accum.reset_index(inplace = True)


In [297]:
accum.to_csv('/cliendiciembre.csv',header=False)